In [3]:
import fastf1
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import General
import re

In [4]:
os.makedirs('cache', exist_ok=True)
fastf1.Cache.enable_cache('cache')

In [5]:
def GetSumDelToWinner(sess):
    if(sess[4]=="Q"):
        return pd.DataFrame()
    df = pd.read_csv(sess)
    df = df.drop(df.columns[0], axis=1)
    if(df.empty):
        return pd.DataFrame()
    LapTimes = []
    for d in df.columns:
        laps = df[d]
        lap_times = []
        lt = laps
        for x in lt:
            if np.isnan(x):
                x = 0
            lap_times.append(x)

        LapTimes.append(lap_times)
    ndf = pd.DataFrame(LapTimes)
    ndf = ndf.transpose()
    ndf.head()


    finalres = df.columns
    winner = finalres[0]
    finalres
    ndf.columns = finalres
    ndf.head()

    dff = df.replace(np.nan, 0)

    def classify_driver(column):
        col_data = dff[column]
        original_col = df[column]
        
        if col_data.eq(0).all() and original_col.isna().all():
            return "DNS"
        
        if original_col.notna().any():
            nan_mask = original_col.isna()
            if nan_mask.any():
                last_valid_idx = original_col.last_valid_index()
                if last_valid_idx is not None:
                    remaining_entries = original_col.iloc[last_valid_idx + 1:]
                    if len(remaining_entries) > 0 and remaining_entries.isna().all():
                        return "DNF"
        
        if original_col.notna().any():
            return "Finished"
        
        return "DNS"

    driver_status = {col: classify_driver(col) for col in dff.columns}
    fin = []
    not_fin = []
    for d in driver_status:
        if driver_status[d]=="Finished":
            fin.append(d)
        else:
            not_fin.append(d)

    fdf = df[fin]
    fdf = fdf.replace(0, np.nan)
    fdf = fdf.apply(lambda col: col.fillna(col.mean()), axis=0)
    fdf.head()


    winner = fdf.columns[0]
    winner_cum = fdf[winner].cumsum()
    delta_to_winner = fdf.cumsum().subtract(winner_cum, axis=0)
    sum_delta = delta_to_winner.sum(skipna=True)

    worst = sum_delta.max()
    dnf_value = 1.5 * worst
    n = len(not_fin)
    dnf_series = pd.Series([dnf_value]*n, index=not_fin)

    sum_delta_final = pd.concat([sum_delta, dnf_series])

    return sum_delta_final


In [6]:
def GetAllSumDel(folder_path = "2024"):
    master_df = pd.DataFrame()
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            m = re.search(r'R(\d+)', file_name)
            if not m:
                continue
            race_no = int(m.group(1))

            file_path = os.path.join(folder_path, file_name)
            file_path = file_path.replace("\\", "/")
            print(file_path)
            
            result_df = GetSumDelToWinner(file_path)            

            result_df.index = result_df.index.astype(int)
            result_df = result_df.sort_index()
            if(result_df.empty == True):
                continue
            master_df[race_no] = result_df

    master_df = master_df.reindex(sorted(master_df.columns), axis=1)
    return master_df


In [7]:
df = GetAllSumDel()
df.head()

2024/R10Spain.csv
2024/R11Austria.csv
2024/R12United Kingdom.csv
2024/R13Hungary.csv
2024/R14Belgium.csv
2024/R15Netherlands.csv
2024/R16Italy.csv
2024/R17Azerbaijan.csv
2024/R18Singapore.csv
2024/R19United States.csv
2024/R1Bahrain.csv
2024/R20Mexico.csv
2024/R21Brazil.csv
2024/R22United States.csv
2024/R23Qatar.csv
2024/R24United Arab Emirates.csv
2024/R2Saudi Arabia.csv
2024/R3Australia.csv
2024/R4Japan.csv
2024/R5China.csv
2024/R6United States.csv
2024/R7Italy.csv
2024/R8Monaco.csv
2024/R9Canada.csv


,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.000,0.000000,4054.3035,0.000000,0.000000,83.629250,0.0000,3148.545,0.000,0.0000,...,637.5880,894.575,879.412846,1142.6040,654.111222,2227.356,0.000000,669.5530,0.000000,1905.343
2,4956.057,8812.284842,NaN,4116.792588,7319.296887,5491.231714,4257.5895,5627.268,4080.696,4279.7475,...,3661.2195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4956.057,8812.284842,4054.3035,4116.792588,11183.395330,3294.308143,4257.5895,5627.268,2026.512,4279.7475,...,3661.2195,2408.806,2528.605213,5510.4825,NaN,NaN,NaN,NaN,NaN,NaN
4,1696.224,1744.307292,488.5960,1083.216882,3757.549796,0.000000,341.5510,1203.572,125.290,188.1390,...,0.0000,369.928,1038.222960,0.0000,819.428778,647.026,617.062373,1157.7680,1996.556588,0.000
10,4956.057,8812.284842,4054.3035,4116.792588,7455.596887,3174.793143,4257.5895,5627.268,2627.984,2105.9600,...,3661.2195,3761.865,2221.931860,5510.4825,6087.218909,4765.686,722.820881,4173.3615,9216.788774,2085.426


In [8]:
df = df.fillna(df.mean())
df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.000,0.000000,4054.303500,0.000000,0.000000,83.629250,0.0000,3148.545,0.000,0.0000,...,637.5880,894.575000,879.412846,1142.604000,654.111222,2227.356000,0.000000,669.553000,0.000000,1905.343000
2,4956.057,8812.284842,2789.390421,4116.792588,7319.296887,5491.231714,4257.5895,5627.268,4080.696,4279.7475,...,3661.2195,2115.610526,2696.614845,4223.409842,5859.664976,3365.038944,3511.232572,2093.324889,9310.296135,3149.736824
3,4956.057,8812.284842,4054.303500,4116.792588,11183.395330,3294.308143,4257.5895,5627.268,2026.512,4279.7475,...,3661.2195,2408.806000,2528.605213,5510.482500,5859.664976,3365.038944,3511.232572,2093.324889,9310.296135,3149.736824
4,1696.224,1744.307292,488.596000,1083.216882,3757.549796,0.000000,341.5510,1203.572,125.290,188.1390,...,0.0000,369.928000,1038.222960,0.000000,819.428778,647.026000,617.062373,1157.768000,1996.556588,0.000000
10,4956.057,8812.284842,4054.303500,4116.792588,7455.596887,3174.793143,4257.5895,5627.268,2627.984,2105.9600,...,3661.2195,3761.865000,2221.931860,5510.482500,6087.218909,4765.686000,722.820881,4173.361500,9216.788774,2085.426000
11,889.041,475.238500,1473.392000,522.140196,1140.298264,2210.043143,2311.0200,5627.268,4080.696,2385.4520,...,1582.8610,1309.015000,5035.593994,5510.482500,2877.590778,4765.686000,3358.530687,2014.084000,14893.222160,5192.784000
14,2550.125,1605.156271,1603.093000,1728.430804,3554.950776,2929.106143,4257.5895,5627.268,1233.095,4279.7475,...,3661.2195,2115.078000,1698.452520,5510.482500,9450.373555,4765.686000,2596.018821,2301.891000,9343.540774,2880.636000
16,1482.913,884.147229,541.580000,1946.480078,1233.750509,390.401357,426.1190,0.000,4080.696,783.2370,...,1110.9110,0.000000,-25.372620,2656.319000,0.000000,516.206000,1158.659433,720.744000,5385.845828,1335.099000
18,3304.038,8812.284842,2323.626000,4116.792588,1060.081679,3327.597143,2838.3930,5627.268,1861.895,4279.7475,...,3661.2195,3761.865000,5035.593994,5510.482500,9450.373555,4765.686000,9005.634839,2782.241000,14893.222160,5192.784000
20,4956.057,5758.328398,4054.303500,4116.792588,5733.848250,3660.821143,4257.5895,5627.268,2579.039,4279.7475,...,3661.2195,1975.342000,2696.614845,5510.482500,3323.305222,2650.449000,3511.232572,2413.758000,9629.046774,5192.784000


In [10]:
df.to_csv("SumDel.csv",index=True)